### Что такое RFM-анализ?
RFM (Recency, Frequency, Monetary) — это метод сегментации пользователей, основанный на трех показателях:  
- **Recency** (давность) — как давно пользователь совершал покупку или был активен.  
- **Frequency** (частота) — как часто пользователь совершает покупки.  
- **Monetary** (стоимость) — сколько пользователь потратил за определенный период.  

RFM помогает понять поведение клиентов и выделить ключевые группы для таргетированных действий.

---

### Код RFM-анализа

#### 1. **Импорт библиотек**
```python
import pandas as pd
import numpy as np
from scipy.stats import f_oneway
import seaborn as sns
import matplotlib.pyplot as plt
```

#### 2. **Расчет RFM-метрик**
```python
# Загрузка данных
data = pd.read_csv('user_data.csv')  # Данные с колонками: user_id, last_purchase_date, purchase_id, purchase_value
data['last_purchase_date'] = pd.to_datetime(data['last_purchase_date'])

# Опорная дата для расчета Recency
reference_date = pd.Timestamp('2024-11-19')

# Группировка данных и расчет RFM
rfm = data.groupby('user_id').agg(
    Recency=('last_purchase_date', lambda x: (reference_date - x.max()).days),
    Frequency=('purchase_id', 'count'),
    Monetary=('purchase_value', 'sum')
).reset_index()
```

#### 3. **Оценка и сегментация пользователей**
```python
# Присвоение оценок по квантилям
rfm['R_score'] = pd.qcut(rfm['Recency'], 5, labels=[5, 4, 3, 2, 1])
rfm['F_score'] = pd.qcut(rfm['Frequency'], 5, labels=[1, 2, 3, 4, 5])
rfm['M_score'] = pd.qcut(rfm['Monetary'], 5, labels=[1, 2, 3, 4, 5])

# Итоговый RFM Score
rfm['RFM_Score'] = rfm[['R_score', 'F_score', 'M_score']].sum(axis=1).astype(int)

# Сегментация на группы
rfm['Segment'] = rfm['RFM_Score'].apply(
    lambda x: 'Champions' if x >= 12 else
              'Loyal Customers' if x >= 9 else
              'Potential Loyalists' if x >= 6 else
              'At Risk' if x >= 4 else 'Lost'
)
```

#### 4. **Анализ сегментов**
```python
# Средние показатели в каждом сегменте
segment_analysis = rfm.groupby('Segment')[['Recency', 'Frequency', 'Monetary']].mean()
print(segment_analysis)

# Пример добавления случайной метрики конверсии
rfm['Conversion'] = np.random.rand(len(rfm)) * 100
```

#### 5. **Проверка гипотезы различий между сегментами**
```python
# ANOVA для сравнения конверсий между сегментами
anova_result = f_oneway(
    *[rfm.loc[rfm['Segment'] == seg, 'Conversion'] for seg in rfm['Segment'].unique()]
)
print(f"ANOVA p-value: {anova_result.pvalue}")
```

#### 6. **Визуализация результатов**
```python
# Визуализация распределения конверсий
sns.boxplot(x='Segment', y='Conversion', data=rfm, order=rfm['Segment'].value_counts().index)
plt.title('Conversion Rates by Segment')
plt.xticks(rotation=45)
plt.show()
```

---

### Выводы
1. **RFM-анализ** позволяет выделить ключевые группы пользователей, например:
   - *Champions* — наиболее активные и прибыльные пользователи.
   - *Lost* — клиенты, которые давно не совершали покупок.
2. **Анализ конверсии** выявляет различия в поведении сегментов. Если p-value < 0.05, различия значимы.
3. **Рекомендации для оптимизации**:
   - Создайте персонализированные предложения для группы *At Risk*.
   - Активно удерживайте *Champions* с помощью бонусов или эксклюзивных акций.